In [34]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
from datetime import datetime
from copy import deepcopy
import json
import matplotlib.pyplot as plt
import seaborn as sns
import os
import geopandas as gpd
import fsspec
import re

from time import sleep

# Mapbox
from mapbox import Geocoder

# Import dataset
aurora_cara = pd.read_csv("Aurora v2.1 data file - caracterización.csv")
aurora_feedback = pd.read_csv("Aurora v2.1 data file - ayudaHumanitaria.csv")
aurora_monitoreo = pd.read_csv("Aurora v2.1 data file - monitoreo.csv")

Creating the panel data from Aurora (first observation of each ID is first connection and the followings, monitorings)

In [35]:
#Merge tables of first connection 
aurora = pd.merge(aurora_cara, aurora_feedback)
aurora.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1137 entries, 0 to 1136
Data columns (total 70 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   ¿Quién entra al sistema?                                1137 non-null   object 
 1   ¿Cómo interactúa con el sistema?                        1137 non-null   object 
 2   Tipo de enganche                                        1137 non-null   object 
 3   UserId                                                  1137 non-null   int64  
 4   InteraID                                                1137 non-null   object 
 5   Inicio interacción                                      1137 non-null   object 
 6   Final interacción                                       0 non-null      float64
 7   Tiempo de interacción                                   333 non-null    float64
 8   Alguien te está ayudando a conectarte 

In [36]:
#Adding the monitorings tables
aurora_comple= aurora.append(aurora_monitoreo)

C:\Users\maola\AppData\Local\Temp\ipykernel_24292\2472287513.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aurora_comple= aurora.append(aurora_monitoreo)


In [37]:
# Drop observations of Aurora team phones, test registers and geographical atypical rows 


user_ids_to_remove = [311571598, 311398466, 311396734, 311361421, 311361350, 311361257, 311337494, 311325070,
                      311325038, 311272934, 310820267, 310543580, 310357249, 310191611, 308421831, 306028996,
                      310191611, 308421831, 306028996, 311725039, 311719001, 311718121, 311699383, 311696700,
                      312179120, 311965863, 311965863, 316773170, 311440316, 313260546, 316563135, 316734459,
                      317064115]

for user_id in user_ids_to_remove:
    aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == user_id].index)

In [38]:
#Variable of date in date format (for generating panel data)
aurora_comple["fecha"] = pd.to_datetime(
aurora_comple["Inicio interacción"], format='%Y-%m-%d %H:%M:%S', errors='coerce', utc=True).dt.strftime('%Y-%m-%d')


In [39]:
# Filling all observations of the same ID with the variable consent and actual country (variables of first connection) 
aurora_comple['País actual'] = aurora_comple.groupby('UserId')['País actual'].transform('first')
aurora_comple['Consentimiento'] = aurora_comple.groupby('UserId')['Consentimiento'].transform('first')
aurora_comple['País destino'] = aurora_comple.groupby('UserId')['País destino'].transform('first')
aurora_comple['Zona país'] = aurora_comple.groupby('UserId')['Zona país'].transform('first')


In [40]:
#filering by consent, cleaning geographical variables that are equal to "None" and drop the ones that can't identify
#the zone of first connection
aurora_comple=aurora_comple[aurora_comple['Consentimiento'] != 'NO'] 
aurora_comple=aurora_comple[aurora_comple['Latitud'] != "None"] 



In [41]:
# Dropping duplicates of monitorings (same answers differents days)
columns_to_consider = [col for col in aurora_comple.columns if col not in ['Inicio interacción', 'InteraID']]

# Use drop_duplicates with the specified columns to consider
aurora_comple = aurora_comple.drop_duplicates(subset=columns_to_consider)

In [42]:

aurora_comple= aurora_comple.sort_values(by=['UserId', 'Inicio interacción'])

# Reset the index for proper grouping
aurora_comple= aurora_comple.reset_index(drop=True)

# Create a new variable "Interaction_Sequence" based on the grouping
aurora_comple['Interaction_Sequence'] = aurora_comple.groupby('UserId').cumcount() + 1

default_value = 999999

In [43]:
#Rename variables 
newColumns = {  
                "¿En qué país naciste?" : 'e08_pais_',
                'Otro país de nacimiento'	: 'e09_otro_p',
                '¿En qué país iniciaste tu viaje actual?'	: 'e10_pais_',
                'Otro país de inicio'	: 'e11_otro_p',
                '¿En qué país vivías hace un año?'	 : 'e12_pais_',
                'Otro país'	: 'e13_otro_p',
}

aurora_comple = aurora_comple.rename(columns=newColumns)

In [44]:
# functions
def loadLocalJsonDoc(filepath, dataProp=''):
    """
    return deserialised json in dictionary

    Parameters
    ----------
    filepath: file location or buffer.
    dataProp: (optional) specified property to access required data
    """
    output = {}
    with open(file=filepath, mode='r', encoding='utf-8') as f:
        json_load = json.load(f)
        if (dataProp):
            output = json_load[dataProp]
        else:
            output = json_load
    return output


def changeCountriesByExpression(country, valueDict: dict[str, str]):
    output = ""
    for key, value in valueDict.items():
        match = re.match(r"^"+key+r".$", country)
        if (match):
            return value

    return output if len(output) else country


def processCountries(countries: list[str], valueDict: dict[str, str]):
    output = []
    for country in countries:
        try:
            new_country = changeCountriesByExpression(
                country=country, valueDict=valueDict)
            output.append(new_country)
        except Exception as e:
            output.append(default_value)
    return output


def getCountriesWithCoordinates(countries: list[str], geo_countries: gpd.GeoDataFrame):
    output = {}
    for country in countries:
        try:
            filtered_country = geo_countries[geo_countries["NAME"].str.lower(
            ) == country].reindex()
            centroidValue = (filtered_country.centroid).iloc[0]
            output[country] = {"x": centroidValue.x, "y": centroidValue.y}
        except Exception as e:
            print(e)
            output[country] = {"x": default_value, "y": default_value}

    return output


def getCoordinate(value: str, side: str, valueDict: dict[str, tuple[int, int]], expressionDict: dict[str, str]):
    try:
        country = changeCountriesByExpression(value, expressionDict)
        return valueDict[country][side]
    except Exception as e:
        return default_value


def processFieldCoordinates(df: pd.DataFrame, columnDict: dict[str, dict[str, str]], valueDict: dict[str, tuple[int, int]], expressionDict: dict[str, str]):
    local_df = deepcopy(df)
    for column in columnDict.keys():
        local_df[columnDict[column]["x"]] = local_df[column].str.lower().apply(
            lambda x: getCoordinate(x, "x", valueDict, expressionDict))
        local_df[columnDict[column]["y"]] = local_df[column].str.lower().apply(
            lambda x: getCoordinate(x, "y", valueDict, expressionDict))

    return local_df

def processGeocodeData(data):
    features = data['features']
    for feature in features:
        id: str = feature['id']
        match = id.startswith("country")
        if (match):
            return (feature['properties']['short_code'], feature["place_name"])

    return "zz"


def getMapboxGeocoder(token:str):
    if(token):
        return Geocoder(access_token=token)
    else: 
        raise Exception("Invalid Token")

def reverseGeocode(longitude: int, latitude: int, token: str):
    mb_geocoder = getMapboxGeocoder(token)
    response = mb_geocoder.reverse(lat=latitude, lon=longitude)
    if (response.status_code == 200):
        data = response.json()
        return data
    else:
        return None


def processReverseGeoding(data: list[tuple[int, int]], token:str):
    output = []
    for lon, lat in data:
        try:
            result = reverseGeocode(lon, lat, token)
            _decoded = processGeocodeData(result)
            output.append(_decoded)
            sleep(1)
        except:
            output.append(("zz", ""))

    return output

def addReverseGeocodedToDataFrame(df: DataFrame, token:str  ):
    local_df = deepcopy(df)
    coordinates = list(zip(list(local_df['longitude'].astype(float).to_list()), list(
    local_df['latitude'].astype(float).to_list())))
    reversed_geocoded_df = processReverseGeoding(coordinates,token)
    local_df["country_code"] = [x[0] for x in reversed_geocoded_df]
    local_df["country_name_eng"] = [x[1] for x in reversed_geocoded_df]
    return local_df

Building the data base of "interacciones en la ruta" - Carto page

In [45]:
# Creating the dataset for interactions across the route

df = pd.DataFrame(aurora_comple, columns = ['UserId', 'Latitud', 'Longitud','Inicio interacción', 'Interaction_Sequence'])



In [46]:
from datetime import datetime, timezone

def toUnixTimestamp(time):
    possible_formats = ["%Y-%m-%d %H:%M:%S.%f+00:00", "%d/%m/%Y %H:%M:%S.%f+00:00"]

    for format_str in possible_formats:
        try:
            # Parse the date with the specified format
            target = datetime.strptime(time, format_str)

            # Convert the datetime to UTC if it doesn't have a timezone
            if target.tzinfo is None:
                target = target.replace(tzinfo=timezone.utc)

            # Calculate the Unix timestamp in milliseconds
            in_seconds = (target - datetime(1970, 1, 1, tzinfo=timezone.utc)).total_seconds()
            in_milliseconds = int(in_seconds) * 1000

            return in_milliseconds
        except ValueError:
            continue

df["timeunix"] = df["Inicio interacción"].apply(toUnixTimestamp)

In [47]:
#renaming variables
newColumns = {
    'UserId' : 'id',
    'Latitud' : 'lat', 
    'Longitud' : 'lon',
    'Inicio interacción' : 'date',

}

df = df.rename(columns=newColumns)

In [48]:
df["date"] = pd.to_datetime(
df["date"], format='%Y-%m-%d %H:%M:%S', errors='coerce', utc=True).dt.strftime('%Y-%m-%d')


In [49]:
#sortig by time
df = df.sort_values(['id', 'timeunix'], ascending=[True, True])
df['idx'] = df.groupby('id').cumcount()

In [50]:
max_sequence_df = df.groupby('id')['Interaction_Sequence'].max().reset_index()
# Merge the result back to the original DataFrame based on 'id'
df = pd.merge(df, max_sequence_df, on='id', how='left', suffixes=('', '_max'))

# Rename the new column to 'max_seq'
df = df.rename(columns={'Interaction_Sequence_max': 'max_seq'})

In [51]:
# adding coordinates value
country_data_path = "simplecache::https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip"

country_df = ""

with fsspec.open(country_data_path) as file:
    country_df = gpd.read_file(file)


df['lon_eng'] = df['lon']
df['lat_eng'] = df['lat']
df['longitude'] = df['lon']
df['latitude'] = df['lat']

MAPBOX_TOKEN = os.environ.get("MAPBOX_TOKEN")
# This is heavy process that takes a while to finish
# should be used sparingly and closer to end processes.
df = addReverseGeocodedToDataFrame(df, MAPBOX_TOKEN)

In [52]:
#creating the structure of the variables pivoted


df['lat_idx'] = 'lat_' + df['idx'].astype(str)
df['lon_idx'] = 'lon_' + df['idx'].astype(str)

lat = df.pivot(index='id', columns='lat_idx', values='lat')
lon = df.pivot(index='id', columns='lon_idx', values='lon')

# Merge the pivoted DataFrames
reshape = pd.concat([lat, lon], axis=1).reset_index()

# Add the other columns
additional_columns = ['timeunix', 'country_name_eng', 'max_seq', 'country_code']
reshape = pd.merge(reshape, df.groupby('id')[additional_columns].first().reset_index(), on='id')

print(reshape)

             id         lat_0        lat_1 lat_10 lat_11       lat_2  \
0     310942422   -20.4027556   -20.402715    NaN    NaN  -20.402808   
1     310947317  -20.40290667          NaN    NaN    NaN         NaN   
2     310951256           NaN          NaN    NaN    NaN         NaN   
3     310953613   -20.4027542  -20.4027542    NaN    NaN         NaN   
4     310986561    -20.402927          NaN    NaN    NaN         NaN   
...         ...           ...          ...    ...    ...         ...   
1062  321587583   -18.3104707          NaN    NaN    NaN         NaN   
1063  321588497  -18.31037331          NaN    NaN    NaN         NaN   
1064  321588675   -18.3103433          NaN    NaN    NaN         NaN   
1065  321590179   -18.3104783          NaN    NaN    NaN         NaN   
1066  321616757   -17.3784135          NaN    NaN    NaN         NaN   

           lat_3 lat_4 lat_5 lat_6  ... lon_4 lon_5 lon_6 lon_7 lon_8 lon_9  \
0     -20.402808   NaN   NaN   NaN  ...   NaN   NaN   Na

In [53]:
#Create additional variables that were calculate in first round
reshape['pais_fin'] = 999999
reshape['dias'] = 999999



In [54]:
#renaming variables according to first round names
newColumns = {
    'lat_0' : 'lat',
    'lat_1' : 'lat_mon2',
    'lat_2' : 'lat_mon3',
    'lat_3' : 'lat_mon4',
    'lat_4' : 'lat_mon5',
    'lat_5' : 'lat_mon6',
    'lat_6' : 'lat_mon7',
    'lat_7' : 'lat_mon8',
    'lat_8' : 'lat_mon9',
    'lat_9' : 'lat_mon10',
    'lat_10': 'lat_mon11',
    'lat_11': 'lat_mon12',
    'lon_0' : 'lon', 
    'lon_1' : 'lon_mon2', 
    'lon_2' : 'lon_mon3', 
    'lon_3' : 'lon_mon4', 
    'lon_4' : 'lon_mon5', 
    'lon_5' : 'lon_mon6', 
    'lon_6' : 'lon_mon7', 
    'lon_7' : 'lon_mon8', 
    'lon_8' : 'lon_mon9', 
    'lon_9' : 'lon_mon10', 
    'lon_10' : 'lon_mon11', 
    'lon_11' : 'lon_mon12', 
    'country_name_eng' : 'country_name',
    'max_seq' : 'max_interaction'

}

reshape = reshape.rename(columns=newColumns)

In [55]:
reshape.fillna(value=999999, inplace=True)


In [56]:
reshape

,id,lat,lat_mon2,lat_mon11,lat_mon12,lat_mon3,lat_mon4,lat_mon5,lat_mon6,lat_mon7,...,lon_mon7,lon_mon8,lon_mon9,lon_mon10,timeunix,country_name,max_interaction,country_code,pais_fin,dias
0,310942422,-20.4027556,-20.402715,999999,999999,-20.402808,-20.402808,999999,999999,999999,...,999999,999999,999999,999999,1697458695000,Chile,4,cl,999999,999999
1,310947317,-20.40290667,999999,999999,999999,999999,999999,999999,999999,999999,...,999999,999999,999999,999999,1697459774000,Chile,1,cl,999999,999999
2,310951256,999999,999999,999999,999999,999999,999999,999999,999999,999999,...,999999,999999,999999,999999,1697460656000,z,3,z,999999,999999
3,310953613,-20.4027542,-20.4027542,999999,999999,999999,999999,999999,999999,999999,...,999999,999999,999999,999999,1697461149000,Chile,2,cl,999999,999999
4,310986561,-20.402927,999999,999999,999999,999999,999999,999999,999999,999999,...,999999,999999,999999,999999,1697468951000,Chile,1,cl,999999,999999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,321587583,-18.3104707,999999,999999,999999,999999,999999,999999,999999,999999,...,999999,999999,999999,999999,1701011656000,Chile,1,cl,999999,999999
1063,321588497,-18.31037331,999999,999999,999999,999999,999999,999999,999999,999999,...,999999,999999,999999,999999,1701011931000,Chile,1,cl,999999,999999
1064,321588675,-18.3103433,999999,999999,999999,999999,999999,999999,999999,999999,...,999999,999999,999999,999999,1701011986000,Chile,1,cl,999999,999999
1065,321590179,-18.3104783,999999,999999,999999,999999,999999,999999,999999,999999,...,999999,999999,999999,999999,1701012456000,Chile,1,cl,999999,999999


In [331]:
#database for Carto
reshape.to_csv('Carto_map.csv', index=False)